<a href="https://colab.research.google.com/github/CristianMorasso/statistical_project/blob/master/Copia_di_statiscal_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Learning Project: ALFA Dataset Anaysis


In [ ]:
!pip install icecream sktime scikit-learn polars numpy
!rm -rf statiscal_project
!git clone https://github.com/CristianMorasso/statistical_project.git
!pwd
# # Connect Drive to get the Dataset
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)
# dataset_dir= '/content/drive/MyDrive/alfa_dataset'


fatal: destination path 'statistical_project' already exists and is not an empty directory.
/content


In [ ]:
dataset_dir = "statistical_project/data"

In [ ]:
# Read the data from the csv files
import numpy as np
import pandas  as pd
import os

dfs = [] # dataframe list
class_dict = {}

class_id = 0
y_time_series = []

for cl  in  os.listdir(dataset_dir):
  path1 = os.path.join(dataset_dir, cl)
  if not os.path.isdir(path1) or cl == "features":
    print(f"{cl} is not a dir or not useful!")
    continue
  class_dict[class_id] = cl
  print(f"Class: {cl}")
  for f in  os.listdir(path1):
    print(f"\t File:{f}")
    pathf = os.path.join(path1, f)
    df = pd.read_csv(pathf)

    # Remove Label field
    label_field = "field.data"
    try:
      df = df.drop(label_field, axis=1)
    except:
      print(f"\t- The {label_field}~ is not in the {f} file")

    # Save the id for the serie
    y_time_series.append(class_id)

    # This would be useless but the fit algorithm needs this
    df.insert(0, "Y", [ class_id for _ in range(len(df))], True)
    dfs.append(df)

  class_id+=1
  print() # Skip line

y_time_series = np.array(y_time_series)

# Checks
from collections import Counter
print(f"Y: {Counter(list(map(lambda x: class_dict[x], y_time_series)))}")

print(f"\nNumber of Time Series: {len(dfs)}")
assert len(dfs) == 46  # We know this from the repo

Class: ailerons
	 File:carbonZ_2018-10-05-14-34-20_2_right_aileron_failure_with_emr_traj.csv
	 File:carbonZ_2018-09-11-14-52-54_left_aileron__right_aileron__failure.csv
	 File:carbonZ_2018-10-05-14-37-22_2_right_aileron_failure.csv
	 File:carbonZ_2018-09-11-17-27-13_2_both_ailerons_failure.csv
	 File:carbonZ_2018-09-11-17-55-30_2_left_aileron_failure.csv
	 File:carbonZ_2018-10-05-14-37-22_3_left_aileron_failure.csv
	 File:carbonZ_2018-09-11-17-55-30_1_right_aileron_failure.csv

Class: elevator
	 File:carbonZ_2018-09-11-15-05-11_1_elevator_failure.csv
	 File:carbonZ_2018-09-11-14-41-51_elevator_failure.csv

Class: rudder
	 File:carbonZ_2018-09-11-15-06-34_2_rudder_right_failure.csv
	 File:carbonZ_2018-09-11-15-06-34_3_rudder_left_failure.csv
	 File:carbonZ_2018-09-11-17-27-13_1_rudder_zero__left_aileron_failure.csv
	 File:carbonZ_2018-09-11-15-06-34_1_rudder_right_failure.csv

README.md is not a dir or not useful!
Class: no_failure
	 File:carbonZ_2018-09-11-14-16-55_no_failure.csv
	- Th

In [ ]:
print("Before: ")
for df in dfs:
  print(df.shape)

# Only use the variables that can be found in all the dataframes
features = []
# Get the biggest set:
# (We are considering that that the bigger set is just a superset and has all the other base features)
for df in dfs:
   features = max(features, df.columns.values, key=lambda x: len(x))
# Then get the set intersection of all of the features sets
features = set(features)
for df in dfs:
  features = features.intersection(set(df.columns.values))

# Apply the feature mask to all the datframe in our dataset
features = list(features)
dfs = list(map(lambda x: x[features],dfs))

print("\nAfter: ")
for df in dfs:
  print(df.shape)

Before: 
(3242, 27)
(4668, 27)
(2899, 27)
(2032, 27)
(1628, 27)
(1928, 27)
(2662, 27)
(1521, 27)
(2568, 27)
(1383, 27)
(1387, 27)
(2869, 27)
(1405, 27)
(670, 27)
(1335, 27)
(865, 27)
(2330, 27)
(1792, 27)
(607, 52)
(1347, 27)
(969, 27)
(1581, 27)
(1454, 27)
(2660, 27)
(2844, 27)
(2252, 27)
(2185, 27)
(2124, 27)
(2263, 27)
(2632, 27)
(2488, 27)
(2282, 27)
(2330, 27)
(2450, 27)
(2292, 27)
(1331, 27)
(1247, 27)
(3137, 27)
(2609, 27)
(2649, 27)
(2328, 27)
(2134, 27)
(2356, 27)
(2646, 27)
(1844, 27)
(1775, 27)

After: 
(3242, 27)
(4668, 27)
(2899, 27)
(2032, 27)
(1628, 27)
(1928, 27)
(2662, 27)
(1521, 27)
(2568, 27)
(1383, 27)
(1387, 27)
(2869, 27)
(1405, 27)
(670, 27)
(1335, 27)
(865, 27)
(2330, 27)
(1792, 27)
(607, 27)
(1347, 27)
(969, 27)
(1581, 27)
(1454, 27)
(2660, 27)
(2844, 27)
(2252, 27)
(2185, 27)
(2124, 27)
(2263, 27)
(2632, 27)
(2488, 27)
(2282, 27)
(2330, 27)
(2450, 27)
(2292, 27)
(1331, 27)
(1247, 27)
(3137, 27)
(2609, 27)
(2649, 27)
(2328, 27)
(2134, 27)
(2356, 27)
(2646, 27)


## Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest

# Concat all the time series in as single dataframe
concatted_dfs = pd.concat(dfs)
y = concatted_dfs["Y"]

# Now remove it again from the concatted dataframe
concatted_dfs = concatted_dfs.drop("Y", axis=1)
# Also remove the time column!
concatted_dfs = concatted_dfs.drop("%time", axis=1)

# Print the number of nans per per each columns
print(concatted_dfs.isnull().sum())
print(f"Length of Concatted Dataframes: {len(concatted_dfs)}")

field.twist.angular.z          145
field.twist.angular.x          145
delta_pitch                     57
field.linear_acceleration.x    294
field.angular_velocity.x       294
delta_x                         59
field.twist.linear.z           145
field.orientation.x            294
field.xtrack_error              57
field.twist.linear.x           145
delta_z                         59
field.twist.angular.y          145
delta_roll                      59
field.alt_error                 57
delta_yaw                       57
field.aspd_error                57
field.orientation.z            294
delta_y                         59
field.linear_acceleration.z    294
field.orientation.y            294
field.angular_velocity.z       294
field.linear_acceleration.y    294
delta_roll_airspeed             60
field.angular_velocity.y       294
field.twist.linear.y           145
dtype: int64
Length of Concatted Dataframes: 96000


In [ ]:
# Inpute the nan values to the mean of the other entries value of the same column
from sklearn.impute import SimpleImputer
import numpy as np

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(concatted_dfs)
inp_out = imp.transform(concatted_dfs)

In [ ]:
# Get the best 20 features of the dataset
columns_ = concatted_dfs.columns.values
feature_mask= SelectKBest(k=20).fit(inp_out, y).get_feature_names_out(columns_)
# Uses: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif

print("Feature Mask:")
print(feature_mask)
print(f"Length of Feature Mask: {len(feature_mask)}")

Feature Mask:
['field.twist.angular.z' 'delta_pitch' 'field.linear_acceleration.x'
 'delta_x' 'field.twist.linear.z' 'field.orientation.x'
 'field.xtrack_error' 'delta_z' 'field.twist.angular.y' 'field.alt_error'
 'delta_yaw' 'field.aspd_error' 'delta_y' 'field.linear_acceleration.z'
 'field.orientation.y' 'field.angular_velocity.z'
 'field.linear_acceleration.y' 'delta_roll_airspeed'
 'field.angular_velocity.y' 'field.twist.linear.y']
Length of Feature Mask: 20


In [ ]:
# Now finally use the mask on every dataframe in the original dataframeList
# and also inpute but using time-series specific data:
new_dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[feature_mask]) , dfs))

# Now we have the 20 best features without any nan value:
type(new_dfs[0]) # is a numpy.ndarray! wow!
print(new_dfs[0][:2,:]) # First 2 lines
print(new_dfs[0].shape) # shape of the array

[[-1.40247777e-01  3.60856056e+00  2.03362941e-01  1.32870197e+01
  -9.80125368e-01 -3.07238990e-02 -4.14283066e+01 -9.24748778e-01
  -4.29745093e-02  1.79999992e-01  1.89663155e+02 -3.45677948e+02
   1.80730991e+01  1.10048680e+01 -5.50597940e-02 -6.21675979e-02
   6.12022871e-01  1.50000000e+01 -1.01050220e-01  2.43065834e+00]
 [-1.40247777e-01  3.61856055e+00  2.03362941e-01  1.31595628e+01
  -9.80125368e-01 -3.07238990e-02 -4.14283066e+01 -8.96695733e-01
  -4.29745093e-02  1.79999992e-01  1.89663155e+02 -3.45677948e+02
   1.80944805e+01  1.10048680e+01 -5.50597940e-02 -6.21675979e-02
   6.12022871e-01  1.60000000e+01 -1.01050220e-01  2.43065834e+00]]
(3242, 20)


In [ ]:
# Now finally use the mask on every dataframe in the original
# dataframeList and also inpute but using time-series specific data:
new_dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[feature_mask]) , dfs))

# Now we have the 20 best features without any nan value:
type(new_dfs[0]) # is a numpy.ndarray! wow!
print(new_dfs[0][:2,:]) # First 2 lines
print(new_dfs[0].shape) # shape of the array

[[-1.40247777e-01  3.60856056e+00  2.03362941e-01  1.32870197e+01
  -9.80125368e-01 -3.07238990e-02 -4.14283066e+01 -9.24748778e-01
  -4.29745093e-02  1.79999992e-01  1.89663155e+02 -3.45677948e+02
   1.80730991e+01  1.10048680e+01 -5.50597940e-02 -6.21675979e-02
   6.12022871e-01  1.50000000e+01 -1.01050220e-01  2.43065834e+00]
 [-1.40247777e-01  3.61856055e+00  2.03362941e-01  1.31595628e+01
  -9.80125368e-01 -3.07238990e-02 -4.14283066e+01 -8.96695733e-01
  -4.29745093e-02  1.79999992e-01  1.89663155e+02 -3.45677948e+02
   1.80944805e+01  1.10048680e+01 -5.50597940e-02 -6.21675979e-02
   6.12022871e-01  1.60000000e+01 -1.01050220e-01  2.43065834e+00]]
(3242, 20)


In [123]:
# Finally let's create a multiIndexed pandas.Dataframe so that we can use our data with the sktime library
_dfs = []
for i, npVals in enumerate(new_dfs):
  df = pd.DataFrame(npVals,columns=feature_mask)
  df.insert(0, "flight", [ i for _ in range(len(df))], True)
  df.insert(1, "step", [ a for a in range(len(df))], True)
  _dfs.append(df)
dfs = pd.concat(_dfs)
print(dfs.shape)
dfs = dfs.drop_duplicates(subset=['flight', 'step'], keep='last')
dfs = dfs.set_index(["flight", "step"])
print(dfs.shape)



(96000, 22)
(96000, 20)


# Testing Models:

In [ ]:
from sktime.datatypes import MTYPE_REGISTER
pd.DataFrame(MTYPE_REGISTER)

,0,1,2
0,pd.Series,Series,pd.Series representation of a univariate series
1,pd.DataFrame,Series,pd.DataFrame representation of a uni- or multi...
2,np.ndarray,Series,"2D numpy.ndarray with rows=samples, cols=varia..."
3,xr.DataArray,Series,xr.DataArray representation of a uni- or multi...
4,dask_series,Series,xdas representation of a uni- or multivariate ...
5,nested_univ,Panel,"pd.DataFrame with one column per variable, pd...."
6,numpy3D,Panel,"3D np.array of format (n_instances, n_columns,..."
7,numpyflat,Panel,"WARNING: only for internal use, not a fully su..."
8,pd-multiindex,Panel,"pd.DataFrame with multi-index (instances, time..."
9,pd-wide,Panel,"pd.DataFrame in wide format, cols = (instance*..."


In [141]:
from sktime.split import temporal_train_test_split
from sktime.datatypes import check_raise
from icecream import ic

# check_raise(y, mtype='np.ndarray')

check_raise(dfs, mtype="pd-multiindex")
check_raise(y_time_series, mtype='np.ndarray')


# X_train, y_train X_test, y_test= temporal_train_test_split(X=dfs,y=y_time_series, test_size=10)

y_train, y_test, X_train, X_test  = temporal_train_test_split(X=dfs,y=y_time_series, test_size=10)
ic(X_train.shape)
ic(y_train.shape)
ic(X_test.shape)
ic(y_test.shape)
X_train.head()


ic| X_train.shape: (73275, 20)
ic| y_train.shape: (36, 1)
ic| X_test.shape: (22725, 20)
ic| y_test.shape: (10, 1)


field.twist.angular.z  delta_pitch  field.linear_acceleration.x  \
flight step                                                                    
0      0                 -0.140248     3.608561                     0.203363   
       1                 -0.140248     3.618561                     0.203363   
       2                 -0.140248     4.408561                     0.203363   
       3                 -0.140248     2.628561                     0.203363   
       4                 -0.140248     2.638561                     0.203363   

               delta_x  field.twist.linear.z  field.orientation.x  \
flight step                                                         
0      0     13.287020             -0.980125            -0.030724   
       1     13.159563             -0.980125            -0.030724   
       2     13.159563             -0.980125            -0.030724   
       3     12.889819             -0.980125            -0.030724   
       4     12.593725             -0.788198            -0.030724   

             field.xtrack_error   delta_z  field.twist.angular.y  \
flight step                                                        
0      0             -41.428307 -0.924749              -0.042975   
       1             -41.428307 -0.896696              -0.042975   
       2            -381.803833 -0.896696              -0.042975   
       3            -380.054047 -0.811358              -0.042975   
       4            -380.054047 -0.722598              -0.042975   

             field.alt_error   delta_yaw  field.aspd_error    delta_y  \
flight step                                                             
0      0                0.18  189.663155       -345.677948  18.073099   
       1                0.18  189.663155       -345.677948  18.094481   
       2                0.28   51.663155       -345.677948  18.094481   
       3                0.37   51.663155       -251.090424  18.107399   
       4                0.37   51.663155       -251.090424  18.126093   

             field.linear_acceleration.z  field.orientation.y  \
flight step                                                     
0      0                       11.004868             -0.05506   
       1                       11.004868             -0.05506   
       2                       11.004868             -0.05506   
       3                       11.004868             -0.05506   
       4                       11.004868             -0.05506   

             field.angular_velocity.z  field.linear_acceleration.y  \
flight step                                                          
0      0                    -0.062168                     0.612023   
       1                    -0.062168                     0.612023   
       2                    -0.062168                     0.612023   
       3                    -0.062168                     0.612023   
       4                    -0.062168                     0.612023   

             delta_roll_airspeed  field.angular_velocity.y  \
flight step                                                  
0      0                    15.0                  -0.10105   
       1                    16.0                  -0.10105   
       2                    16.0                  -0.10105   
       3                    16.0                  -0.10105   
       4                    16.0                  -0.10105   

             field.twist.linear.y  
flight step                        
0      0                 2.430658  
       1                 2.430658  
       2                 2.430658  
       3                 2.430658  
       4                 3.180451

## NearestNeighbours

In [143]:
# 5-nearest neighbour classifier with Euclidean distance (on flattened time series)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=5, distance=eucl_dist)

# Train the classifier
clf.fit(X_train, y_train)

# Predict labels on new data
y_pred = clf.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sktime/base/_base_panel.py:250: UserWarning: Data seen by KNeighborsTimeSeriesClassifier instance has unequal length series, but this KNeighborsTimeSeriesClassifier instance cannot handle unequal length series. Calls with unequal length series may result in error or unreliable results.
  warn(msg, obj=self)
/usr/local/lib/python3.10/dist-packages/sktime/base/_base_panel.py:250: UserWarning: Data seen by KNeighborsTimeSeriesClassifier instance has unequal length series, but this KNeighborsTimeSeriesClassifier instance cannot handle unequal length series. Calls with unequal length series may result in error or unreliable results.
  warn(msg, obj=self)


ValueError: cannot reshape array of size 454500 into shape (10,3137,20)